In [1]:
import pandas as pd
import openpyxl 
import datetime

user_india = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Attendance Report (New)_india.xlsx") 
user_nepal = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Attendance Report (New)_nepal.xlsx")

master_position_india=pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Master Data (Position)_india.xlsx") 
master_position_nepal=pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Master Data (Position)_nepal.xlsx")

#sod_india = pd.read_excel(r"C:\Users\Admin\Downloads\Secondary Order Dump.xlsx") 
sod_india = pd.read_csv(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Secondary Order Dump (New)_india.csv") 
sod_nepal = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Secondary Order Dump(New)_nepal.xlsx")

sod_nepal.columns = sod_nepal.columns.str.strip()
sod_india.columns = sod_india.columns.str.strip()

user = pd.concat([user_india, user_nepal], ignore_index=True)
master_position = pd.concat([master_position_india, master_position_nepal], ignore_index=True)
master_position_s3 = pd.concat([master_position_india, master_position_nepal], ignore_index=True)

master_position=master_position.drop_duplicates(subset='Beats Erp Id')

sod = pd.concat([sod_india, sod_nepal], ignore_index=True)

#db_master = pd.read_excel(r"C:\Users\Shree\OneDrive - Niine Pvt. Ltd\Documents\Shreemala\SR_Tracker_Code\upload_filesuct Master\DB Master.xlsx",sheet_name='DB Master')

target = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\India_Target_Template - Nov'24.xlsx",sheet_name='DSR & USR Tgts',header=3)

target_territory = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\India_Target_Template - Nov'24.xlsx",sheet_name='Territory Tgts',header=4)
target_zone = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\India_Target_Template - Nov'24.xlsx",sheet_name='ZoneTargets',header=2) 
target_region = pd.read_excel(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\India_Target_Template - Nov'24.xlsx",sheet_name='Region Tgts',header=1)

C:\Users\Agastya Singh\AppData\Local\Temp\ipykernel_12784\4291339300.py:12: DtypeWarning: Columns (30,33,34,50,62,67,68,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  sod_india = pd.read_csv(r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\Secondary Order Dump (New)_india.csv")
C:\Users\Agastya Singh\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [2]:
sod['Order Date'].unique()

array(['02-11-2024', '01-11-2024', '04-11-2024', '03-11-2024',
       '05-11-2024', '06-11-2024', '07-11-2024', '08-11-2024',
       '09-11-2024', '10-11-2024', '11-11-2024', '12-11-2024',
       '13-11-2024', '14-11-2024', '15-11-2024', '16-11-2024',
       '18-11-2024', '17-11-2024', Timestamp('2024-11-04 00:00:00'),
       Timestamp('2024-11-03 00:00:00'), Timestamp('2024-11-06 00:00:00'),
       Timestamp('2024-11-05 00:00:00'), Timestamp('2024-11-08 00:00:00'),
       Timestamp('2024-11-07 00:00:00'), Timestamp('2024-11-10 00:00:00'),
       Timestamp('2024-11-12 00:00:00'), Timestamp('2024-11-11 00:00:00'),
       Timestamp('2024-11-13 00:00:00'), Timestamp('2024-11-14 00:00:00'),
       Timestamp('2024-11-15 00:00:00'), Timestamp('2024-11-17 00:00:00'),
       Timestamp('2024-11-18 00:00:00')], dtype=object)

In [3]:
def format_date(date):
    if isinstance(date, pd.Timestamp):
        return date.strftime('%d-%m-%Y')
    elif isinstance(date, str):
        return date  # Assume it's already in the correct format
    else:
        return None  # Handle unexpected types if necessary

# Apply the function to the 'order_date' column
sod['Order Date'] = sod['Order Date'].apply(format_date)

In [4]:
users=user[(user['User Designation'] == 'USR') | (user['User Designation'] == 'DSR') | (user['User Position Level'] == 'L1Position') | (user['User Position Level'] == 'L2Position') ] 
users = users.rename(columns=lambda x: x.strip())
users=users[['Level 5 Position','Level 4 Position','L4Position User','Level 3 Position','L3Position User','Salesman ErpId', 'User', 'User Status','Date of Joining']] 
users = users.rename(columns={'Level 3 Position': 'L3 Position Name'})
users = users.rename(columns={'Salesman ErpId': 'User Erp Id'})

master_position = master_position.rename(columns={'Zone': 'Regions'})
master_position = master_position.rename(columns={'Region': 'Zone'})
master_position = master_position.rename(columns=lambda x: x.strip())
master_position=master_position[['L3 Position Name','L2 Position Name','L1 Position Name','User Erp Id','Zone','Regions','Territory','Beats','Beats Erp Id','SuperStockist Erp Id','SuperStockist Name','Distributor Name', 'Distributor Erp Id','Count of Outlets']]

se_level_master=master_position[['Zone','Regions','Territory','L3 Position Name']] 
se_level_master=se_level_master.drop_duplicates(subset='L3 Position Name')
sr_level_master=master_position[['L1 Position Name','User Erp Id','Beats','Beats Erp Id','SuperStockist Erp Id','SuperStockist Name','Distributor Name', 'Distributor Erp Id','Count of Outlets']]
#sr_level_master=sr_level_master.drop_duplicates(subset='User Erp Id')

sr_master = pd.merge(users, se_level_master, on='L3 Position Name', how='left')
sr_master_nan_territory = sr_master[sr_master['L3 Position Name'].isna()]

sr_master_nan_territory=sr_master_nan_territory[['Zone','Regions','Territory','User Erp Id']]
sr_master_nan_territory = sr_master_nan_territory.drop_duplicates()
se_level_master=master_position[['Zone','Regions','Territory','User Erp Id']]
se_level_master = se_level_master.drop_duplicates()
sr_master_nan_territory = pd.merge(sr_master_nan_territory, se_level_master, on='User Erp Id', how='left')
sr_master_nan_territory=sr_master_nan_territory[['Zone_y','Regions_y','Territory_y','User Erp Id']]
sr_master_nan_territory = sr_master_nan_territory.drop_duplicates()

sr_master_nan_territory = sr_master_nan_territory.dropna() 
sr_master = pd.merge(sr_master, sr_master_nan_territory, on='User Erp Id', how='left')
sr_master.loc[sr_master['L3 Position Name'].isna(), 'Territory'] = sr_master['Territory_y']
sr_master.loc[sr_master['L3 Position Name'].isna(), 'Zone'] = sr_master['Zone_y']
sr_master.loc[sr_master['L3 Position Name'].isna(), 'Regions'] = sr_master['Regions_y']

sr_master = pd.merge(sr_master, sr_level_master, on='User Erp Id', how='outer')
#sr_master.to_excel(r'C:\Users\Admin\Downloads\sr_master.xlsx')
#sr_master = sr_master.dropna(subset=['Territory'])
sod = sod.drop_duplicates()
sod = sod.rename(columns={'ESM Erp Id': 'Erp Code'})


beat_retailing = sod.groupby(['Erp Code','Beat Erp Id'])['Order Date'].nunique().reset_index()
beat_tc = sod.groupby(['Beat Erp Id'])['Order No'].nunique().reset_index()
beat_utc = sod.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
condition = sod['Net Value'] > 0
sod_prod = sod[condition]
beat_pc = sod_prod.groupby(['Beat Erp Id'])['Order No'].nunique().reset_index()
beat_upc = sod_prod.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
beat_lpc = sod_prod.groupby(['Beat Erp Id'])['Product ErpId'].count().reset_index()


napkin = sod[sod['PrimaryCategory'] == 'Sanitary Pads']
diaper = sod[sod['PrimaryCategory'] == 'Diaper']

beat_napkin_ach = napkin.groupby(['Beat Erp Id'])['Net Value'].sum().reset_index()
beat_diaper_ach = diaper.groupby(['Beat Erp Id'])['Net Value'].sum().reset_index()
beat_napkin_eco = napkin.groupby(['Beat Erp Id'])['Order No'].nunique().reset_index()
beat_diaper_eco = diaper.groupby(['Beat Erp Id'])['Order No'].nunique().reset_index()

beat_total_ach = sod.groupby(['Beat Erp Id'])['Net Value'].sum().reset_index()

eco_dc = sod[sod['SecondaryCategory'] == 'Dry Comfort']
eco_ns = sod[sod['SecondaryCategory'] == 'Naturally Soft']
eco_ultra = sod[sod['SecondaryCategory'] == 'Ultra']
eco_ultrapremium = sod[sod['SecondaryCategory'] == 'Ultra Premium']
eco_babywipes = sod[sod['SecondaryCategory'] == 'Baby Wipes']

eco_dc_beat = eco_dc.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
eco_ns_beat = eco_ns.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
eco_ultra_beat = eco_ultra.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
eco_ultrapremium_beat = eco_ultrapremium.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()
eco_babywipes_beat = eco_babywipes.groupby(['Beat Erp Id'])['Outlet Erp Id'].nunique().reset_index()


beat_tc = beat_tc.rename(columns={'Order No': 'TC'})
beat_utc = beat_utc.rename(columns={'Outlet Erp Id': 'UTC'})
beat_pc = beat_pc.rename(columns={'Order No': 'PC'})
beat_upc = beat_upc.rename(columns={'Outlet Erp Id': 'UPC'})
beat_lpc = beat_lpc.rename(columns={'Product ErpId': 'LPC'})
beat_napkin_ach = beat_napkin_ach.rename(columns={'Net Value': 'Napkin_ach'})
beat_diaper_ach = beat_diaper_ach.rename(columns={'Net Value': 'Diaper_ach'})
beat_napkin_eco = beat_napkin_eco.rename(columns={'Order No': 'Napkin_eco'})
beat_diaper_eco = beat_diaper_eco.rename(columns={'Order No': 'Diaper_eco'})
eco_dc_beat = eco_dc_beat.rename(columns={'Outlet Erp Id': 'dc_eco'})
eco_ns_beat = eco_ns_beat.rename(columns={'Outlet Erp Id': 'ns_eco'})
eco_ultra_beat = eco_ultra_beat.rename(columns={'Outlet Erp Id': 'ultra_eco'})
eco_ultrapremium_beat = eco_ultrapremium_beat.rename(columns={'Outlet Erp Id': 'ultrapremium_eco'})
eco_babywipes_beat = eco_babywipes_beat.rename(columns={'Outlet Erp Id': 'babywipes_eco'})



In [5]:
sr_master.columns

Index(['Level 5 Position', 'Level 4 Position', 'L4Position User',
       'L3 Position Name', 'L3Position User', 'User Erp Id', 'User',
       'User Status', 'Date of Joining', 'Zone', 'Regions', 'Territory',
       'Zone_y', 'Regions_y', 'Territory_y', 'L1 Position Name', 'Beats',
       'Beats Erp Id', 'SuperStockist Erp Id', 'SuperStockist Name',
       'Distributor Name', 'Distributor Erp Id', 'Count of Outlets'],
      dtype='object')

In [6]:

sr_master = sr_master.rename(columns={'User Erp Id': 'Erp Code'})
sr_master = sr_master.rename(columns={'Beats Erp Id': 'Beat Erp Id'})

beat_retailing = beat_retailing.drop(columns=['Beat Erp Id'])
beat_retailing=beat_retailing.drop_duplicates(subset='Erp Code')
beat_summary = pd.merge(sr_master, beat_retailing, on='Erp Code', how='left')
beat_summary = pd.merge(beat_summary, beat_tc, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_utc, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_pc, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_upc, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_lpc, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_napkin_ach, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_diaper_ach, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_total_ach, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, eco_dc_beat, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, eco_ns_beat, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, eco_ultra_beat, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, eco_ultrapremium_beat, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, eco_babywipes_beat, on='Beat Erp Id', how='left')

beat_summary = pd.merge(beat_summary, beat_napkin_eco, on='Beat Erp Id', how='left')
beat_summary = pd.merge(beat_summary, beat_diaper_eco, on='Beat Erp Id', how='left')


###count weekdays till today####
def count_working_days_until_today():
    working_days = 0
    today = datetime.date.today()
    current_date = datetime.date(today.year, today.month, 1)  # First day of the month
    while current_date <= today:
        if current_date.weekday() != 5:  # Saturday
            working_days += 1
        current_date += datetime.timedelta(days=1)
    return working_days

number_of_working_days = count_working_days_until_today()-1

###count total weedays in current month #####
def count_weekdays_in_current_month():
    weekdays = 0
    today = datetime.date.today()
    current_date = datetime.date(today.year, today.month, 1)  # First day of the month
    while current_date.month == today.month:
        if current_date.weekday() != 5:  # Saturday
            weekdays += 1
        current_date += datetime.timedelta(days=1)
    return weekdays

number_of_weekdays = count_weekdays_in_current_month()
balance_days=number_of_weekdays-number_of_working_days

beat_summary['Napkin_ach'] = beat_summary['Napkin_ach'] / 100000
beat_summary['Diaper_ach'] = beat_summary['Diaper_ach'] / 100000
beat_summary['Net Value'] = beat_summary['Net Value'] / 100000



In [7]:
def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
        
beat_summary['UTC%'] = beat_summary.apply(lambda row: safe_divide(row['UTC'], row['Count of Outlets']), axis=1)
beat_summary['UPC%'] = beat_summary.apply(lambda row: safe_divide(row['UPC'], row['Count of Outlets']), axis=1)
beat_summary['Average Order value'] = beat_summary.apply(lambda row: safe_divide(row['Net Value'], row['PC']), axis=1)
beat_summary['LPC_value'] = beat_summary.apply(lambda row: safe_divide(row['LPC'], row['PC']), axis=1)                                                        
#sr_summary['TotalAch%'] = sr_summary.apply(lambda row: safe_divide(row['Net Value'], row['FA_total_target']), axis=1)
beat_summary['DRR'] = beat_summary.apply(lambda row: safe_divide(row['Net Value'], number_of_working_days), axis=1)
#sr_summary['RRR'] = sr_summary.apply(lambda row: safe_divide(row['rrr_diff'], balance_days), axis=1)
beat_summary['napkin_eco%'] = beat_summary.apply(lambda row: safe_divide(row['Napkin_eco'], row['Count of Outlets']), axis=1)
beat_summary['diaper_eco%'] = beat_summary.apply(lambda row: safe_divide(row['Diaper_eco'], row['Count of Outlets']), axis=1)                                                        
beat_summary['Average Order value'] *= 100000
beat_summary['Diaper_ach'] = beat_summary['Diaper_ach'].round(2)
beat_summary['Napkin_ach'] = beat_summary['Napkin_ach'].round(2)
beat_summary['UTC%'] = beat_summary['UTC%'].round(2)
beat_summary['UPC%'] = beat_summary['UPC%'].round(2)
#sr_summary['rrr_diff'] = sr_summary['rrr_diff'].round(2)
#sr_summary['TotalAch%'] = sr_summary['TotalAch%'].round(2)
beat_summary['DRR'] = beat_summary['DRR'].round(2)
#sr_summary['RRR'] = sr_summary['RRR'].round(2)
beat_summary['napkin_eco%'] = beat_summary['napkin_eco%'].round(2)
beat_summary['diaper_eco%'] = beat_summary['diaper_eco%'].round(2)
beat_summary['Napkin_ach'] = beat_summary['Napkin_ach'].round(2)

beat_summary['Average Order value'] = beat_summary['Average Order value'].round(2)
beat_summary = beat_summary.dropna(subset=['Erp Code'])
beat_summary = beat_summary.dropna(subset=['User'])

weekdays = count_working_days_until_today()-1


In [8]:

sr_sod=sod[['Order No','Order Date',	'Outlet Erp Id',	'Net Value',	'PrimaryCategory',	'Erp Code',	'SecondaryCategory','Product ErpId']]

sr_master['User Status'] = sr_master['User Status'].fillna('Active')
sr_master2=sr_master[['Erp Code','User','User Status',	'Date of Joining',	'Zone',	'Regions',	'Territory','Count of Outlets']]
sr_master2['Count of Outlets'] = sr_master2['Count of Outlets'].fillna(0)
sr_master_outlets = sr_master2.groupby(['Erp Code','User',	'User Status',	'Date of Joining',	'Zone',	'Regions',	'Territory'])['Count of Outlets'].sum().reset_index()

sr_retailing = sr_sod.groupby(['Erp Code'])['Order Date'].nunique().reset_index()
sr_tc = sr_sod.groupby(['Erp Code'])['Order No'].nunique().reset_index()
sr_utc = sr_sod.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
condition = sr_sod['Net Value'] > 0
sod_prod = sr_sod[condition]
sr_pc = sod_prod.groupby(['Erp Code'])['Order No'].nunique().reset_index()
sr_upc = sod_prod.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
sr_lpc = sod_prod.groupby(['Erp Code'])['Product ErpId'].count().reset_index()


sr_napkin = sr_sod[sr_sod['PrimaryCategory'] == 'Sanitary Pads']
sr_diaper = sr_sod[sr_sod['PrimaryCategory'] == 'Diaper']

sr_napkin_ach = sr_napkin.groupby(['Erp Code'])['Net Value'].sum().reset_index()
sr_diaper_ach = sr_diaper.groupby(['Erp Code'])['Net Value'].sum().reset_index()
sr_napkin_eco = sr_napkin.groupby(['Erp Code'])['Order No'].nunique().reset_index()
sr_diaper_eco = sr_diaper.groupby(['Erp Code'])['Order No'].nunique().reset_index()

sr_total_ach = sr_sod.groupby(['Erp Code'])['Net Value'].sum().reset_index()

sr_eco_dc = sr_sod[sr_sod['SecondaryCategory'] == 'Dry Comfort']
sr_eco_ns = sr_sod[sr_sod['SecondaryCategory'] == 'Naturally Soft']
sr_eco_ultra = sr_sod[sr_sod['SecondaryCategory'] == 'Ultra']
sr_eco_ultrapremium = sr_sod[sr_sod['SecondaryCategory'] == 'Ultra Premium']
sr_eco_babywipes = sr_sod[sr_sod['SecondaryCategory'] == 'Baby Wipes']

sr_eco_dc_beat = sr_eco_dc.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
sr_eco_ns_beat = sr_eco_ns.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
sr_eco_ultra_beat = sr_eco_ultra.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
sr_eco_ultrapremium_beat = sr_eco_ultrapremium.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()
sr_co_babywipes= sr_eco_babywipes.groupby(['Erp Code'])['Outlet Erp Id'].nunique().reset_index()


sr_tc = sr_tc.rename(columns={'Order No': 'TC'})
sr_utc = sr_utc.rename(columns={'Outlet Erp Id': 'UTC'})
sr_pc = sr_pc.rename(columns={'Order No': 'PC'})
sr_upc = sr_upc.rename(columns={'Outlet Erp Id': 'UPC'})
sr_lpc = sr_lpc.rename(columns={'Product ErpId': 'LPC'})
sr_napkin_ach = sr_napkin_ach.rename(columns={'Net Value': 'Napkin_ach'})
sr_diaper_ach = sr_diaper_ach.rename(columns={'Net Value': 'Diaper_ach'})
sr_napkin_eco = sr_napkin_eco.rename(columns={'Order No': 'Napkin_eco'})
sr_diaper_eco = sr_diaper_eco.rename(columns={'Order No': 'Diaper_eco'})
sr_eco_dc_beat = sr_eco_dc_beat.rename(columns={'Outlet Erp Id': 'dc_eco'})
sr_eco_ns_beat = sr_eco_ns_beat.rename(columns={'Outlet Erp Id': 'ns_eco'})
sr_eco_ultra_beat = sr_eco_ultra_beat.rename(columns={'Outlet Erp Id': 'ultra_eco'})
sr_eco_ultrapremium_beat = sr_eco_ultrapremium_beat.rename(columns={'Outlet Erp Id': 'ultrapremium_eco'})
sr_eco_babywipes = sr_eco_babywipes.rename(columns={'Outlet Erp Id': 'babywipes_eco'})

users = users.rename(columns={'User Erp Id': 'Erp Code'})

sr_summary = pd.merge(sr_master_outlets, sr_retailing, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_tc, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_utc, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_pc, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_upc, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_lpc, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_napkin_ach, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_diaper_ach, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_total_ach, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_eco_dc_beat, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_eco_ns_beat, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_eco_ultra_beat, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_eco_ultrapremium_beat, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_eco_babywipes, on='Erp Code', how='left')

sr_summary = pd.merge(sr_summary, sr_napkin_eco, on='Erp Code', how='left')
sr_summary = pd.merge(sr_summary, sr_diaper_eco, on='Erp Code', how='left')

sr_summary=sr_summary[['Erp Code', 'User', 'User Status', 'Date of Joining', 'Zone', 'Regions',
       'Territory', 'Count of Outlets', 'Order Date_x', 'TC', 'UTC', 'PC',
       'UPC','LPC', 'Napkin_ach', 'Diaper_ach', 'Net Value_x', 'dc_eco', 'ns_eco',
       'ultra_eco', 'ultrapremium_eco', 'Order No','Napkin_eco', 'Diaper_eco']]

sr_summary['Napkin_ach'] = sr_summary['Napkin_ach'] / 100000
sr_summary['Diaper_ach'] = sr_summary['Diaper_ach'] / 100000

target = target.rename(columns={'Target': 'FA_napkin_target'})
target = target.rename(columns={'Target.1': 'FA_diaper_target'})
target = target.rename(columns={'Target.2': 'FA_total_target'})
target = target.rename(columns={'Target.3': 'SNS_napkin_target'})
target = target.rename(columns={'Target.4': 'SNS_diaper_target'})
target = target.rename(columns={'Target.5': 'SNS_total_target'})
target = target.rename(columns={'Target.6': 'collection_target'})
target = target.rename(columns={'Target.7': 'nb_focus_target'})
target = target.rename(columns={'ERP Code': 'Erp Code'})
target_fa_sr = target.groupby(['Erp Code'])[['FA_napkin_target', 'FA_diaper_target', 'FA_total_target']].sum().reset_index()
sr_summary = pd.merge(sr_summary, target_fa_sr, on='Erp Code', how='left')
sr_summary = sr_summary.rename(columns={'Net Value_x': 'Total Achivement'})
sr_summary['Total Achivement'] = sr_summary['Total Achivement'] / 100000
sr_summary['rrr_diff']=sr_summary['FA_total_target']-sr_summary['Total Achivement']

def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
        
sr_summary['Diaper Ach%'] = sr_summary.apply(lambda row: safe_divide(row['Diaper_ach'], row['FA_diaper_target']), axis=1)
sr_summary['Napkin Ach%'] = sr_summary.apply(lambda row: safe_divide(row['Napkin_ach'], row['FA_napkin_target']), axis=1)
sr_summary['UTC%'] = sr_summary.apply(lambda row: safe_divide(row['UTC'], row['Count of Outlets']), axis=1)
sr_summary['UPC%'] = sr_summary.apply(lambda row: safe_divide(row['UPC'], row['Count of Outlets']), axis=1)
sr_summary['Average Order value'] = sr_summary.apply(lambda row: safe_divide(row['Total Achivement'], row['PC']), axis=1)
sr_summary['LPC_value'] = sr_summary.apply(lambda row: safe_divide(row['LPC'], row['PC']), axis=1)
sr_summary['TotalAch%'] = sr_summary.apply(lambda row: safe_divide(row['Total Achivement'], row['FA_total_target']), axis=1)
sr_summary['DRR'] = sr_summary.apply(lambda row: safe_divide(row['Total Achivement'], number_of_working_days), axis=1)
sr_summary['RRR'] = sr_summary.apply(lambda row: safe_divide(row['rrr_diff'], balance_days), axis=1)
sr_summary['napkin_eco%'] = sr_summary.apply(lambda row: safe_divide(row['Napkin_eco'], row['Count of Outlets']), axis=1)
sr_summary['diaper_eco%'] = sr_summary.apply(lambda row: safe_divide(row['Diaper_eco'], row['Count of Outlets']), axis=1)                                                        
sr_summary['Average Order value'] *= 100000


C:\Users\Agastya Singh\AppData\Local\Temp\ipykernel_12784\4194916014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sr_master2['Count of Outlets'] = sr_master2['Count of Outlets'].fillna(0)


In [9]:

sr_summary['Diaper_ach'] = sr_summary['Diaper_ach'].round(2)
sr_summary['Napkin_ach'] = sr_summary['Napkin_ach'].round(2)
sr_summary['Napkin Ach%'] = sr_summary['Napkin Ach%'].round(2)
sr_summary['Diaper Ach%'] = sr_summary['Diaper Ach%'].round(2)
sr_summary['UTC%'] = sr_summary['UTC%'].round(2)
sr_summary['UPC%'] = sr_summary['UPC%'].round(2)
sr_summary['rrr_diff'] = sr_summary['rrr_diff'].round(2)
sr_summary['TotalAch%'] = sr_summary['TotalAch%'].round(2)
sr_summary['DRR'] = sr_summary['DRR'].round(2)
sr_summary['RRR'] = sr_summary['RRR'].round(2)
sr_summary['napkin_eco%'] = sr_summary['napkin_eco%'].round(2)
sr_summary['diaper_eco%'] = sr_summary['diaper_eco%'].round(2)
sr_summary['Average Order value'] = sr_summary['Average Order value'].round(2)
weekdays = count_working_days_until_today()-1

sr_summary = sr_summary[sr_summary['User Status'] == 'Active']



In [10]:
territory_data = sr_summary.groupby(['Regions','Zone','Territory']).agg({
    'Erp Code': 'nunique',  # Count unique salesmen
    'Order Date_x': 'sum',
    'Count of Outlets': 'sum' ,
    'TC': 'sum' ,
    'UTC': 'sum' ,
    'PC': 'sum' ,
    'UPC': 'sum' ,
    'LPC_value': 'mean' ,
    'Total Achivement': 'sum',
    'Diaper_ach': 'sum',
    'Napkin_ach': 'sum',
    'Napkin_eco': 'sum',
    'Diaper_eco': 'sum',
    'dc_eco' : 'sum',
    'ns_eco': 'sum',
    'ultra_eco': 'sum',
    'ultrapremium_eco': 'sum',
    'DRR': 'sum' ,
    'RRR': 'sum' 
}).reset_index()




target_territory = target_territory.rename(columns={'Napkin Target': 'Primary_napkin_target'})
target_territory = target_territory.rename(columns={'Diaper Target': 'Primary_diaper_target'})
target_territory = target_territory.rename(columns={'Total Target': 'Primary_total_target'})
target_territory = target_territory.rename(columns={'Napkin Target.1': 'FA_napkin_target'})
target_territory = target_territory.rename(columns={'Diaper Target.1': 'FA_diaper_target'})
target_territory = target_territory.rename(columns={'Total Target .1': 'FA_total_target'})
target_territory = target_territory.rename(columns={'Napkin Target.2': 'SNS_napkin_target'})
target_territory = target_territory.rename(columns={'Diaper Target.2': 'SNS_diaper_target'})
target_territory = target_territory.rename(columns={'Total Target .2': 'SNS_total_target'})

target_territory= target_territory[['Territory','FA_napkin_target', 'FA_diaper_target',

       'FA_total_target','SNS_napkin_target', 'SNS_diaper_target',

       'SNS_total_target']]

 
territory_data = pd.merge(territory_data, target_territory, on='Territory', how='left')

def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
        
        
territory_data['Diaper Ach%'] = territory_data.apply(lambda row: safe_divide(row['Diaper_ach'], row['FA_diaper_target']), axis=1)
territory_data['Napkin Ach%'] = territory_data.apply(lambda row: safe_divide(row['Napkin_ach'], row['FA_napkin_target']), axis=1)
territory_data['UTC%'] = territory_data.apply(lambda row: safe_divide(row['UTC'], row['Count of Outlets']), axis=1)
territory_data['UPC%'] = territory_data.apply(lambda row: safe_divide(row['UPC'], row['Count of Outlets']), axis=1)
territory_data['Average Order value'] = territory_data.apply(lambda row: safe_divide(row['Total Achivement'], row['PC']), axis=1)
territory_data['TotalAch%'] = territory_data.apply(lambda row: safe_divide(row['Total Achivement'], row['FA_total_target']), axis=1)
territory_data['DRR'] = territory_data.apply(lambda row: safe_divide(row['Total Achivement'], number_of_working_days), axis=1)
territory_data['napkin_eco%'] = territory_data.apply(lambda row: safe_divide(row['Napkin_eco'], row['Count of Outlets']), axis=1)
territory_data['diaper_eco%'] = territory_data.apply(lambda row: safe_divide(row['Diaper_eco'], row['Count of Outlets']), axis=1)                                                        
territory_data['Average Order value'] *= 100000
territory_data['Diaper Ach'] = territory_data['Diaper_ach'].round(2)
territory_data['Napkin Ach'] = territory_data['Napkin_ach'].round(2)
territory_data['Napkin Ach%'] = territory_data['Napkin Ach%'].round(2)
territory_data['Diaper Ach%'] = territory_data['Diaper Ach%'].round(2)
territory_data['UTC%'] = territory_data['UTC%'].round(2)
territory_data['UPC%'] = territory_data['UPC%'].round(2)
territory_data['TotalAch%'] = territory_data['TotalAch%'].round(2)
territory_data['DRR'] = territory_data['DRR'].round(2)
territory_data['RRR'] = territory_data['RRR'].round(2)
territory_data['napkin_eco%'] = territory_data['napkin_eco%'].round(2)
territory_data['diaper_eco%'] = territory_data['diaper_eco%'].round(2)
territory_data['Average Order value'] = territory_data['Average Order value'].round(2)
weekdays = count_working_days_until_today()-1



zone_data = sr_summary.groupby(['Regions','Zone']).agg({
    'Erp Code': 'nunique',  # Count unique salesmen
    'Order Date_x': 'sum',
    'Count of Outlets': 'sum' ,
    'TC': 'sum' ,
    'UTC': 'sum' ,
    'PC': 'sum' ,
    'UPC': 'sum' ,
    'LPC_value': 'mean' ,
    'Total Achivement': 'sum',
    'Diaper_ach': 'sum',
    'Napkin_ach': 'sum',
    'Napkin_eco': 'sum',
    'Diaper_eco': 'sum',
    'dc_eco' : 'sum',
    'ns_eco': 'sum',
    'ultra_eco': 'sum',
    'ultrapremium_eco': 'sum',
    'DRR': 'sum' ,
    'RRR': 'sum' 
}).reset_index()


target_zone = target_zone.rename(columns={'Target': 'FA_napkin_target'})
target_zone = target_zone.rename(columns={'Napkin Target': 'Primary_napkin_target'})
target_zone = target_zone.rename(columns={'Diaper Target': 'Primary_diaper_target'})
target_zone = target_zone.rename(columns={'Total Target': 'Primary_total_target'})
target_zone = target_zone.rename(columns={'Napkin Target.1': 'FA_napkin_target'})
target_zone = target_zone.rename(columns={'Diaper Target.1': 'FA_diaper_target'})
target_zone = target_zone.rename(columns={'Total Target .1': 'FA_total_target'})
target_zone = target_zone.rename(columns={'Napkin Target.2': 'SNS_napkin_target'})
target_zone = target_zone.rename(columns={'Diaper Target.2': 'SNS_diaper_target'})
target_zone = target_zone.rename(columns={'Total Target .2': 'SNS_total_target'})

target_zone= target_zone[['Zone','FA_napkin_target', 'FA_diaper_target',

       'FA_total_target','SNS_napkin_target', 'SNS_diaper_target',

       'SNS_total_target']]


zone_data = pd.merge(zone_data, target_zone, on='Zone', how='left')


def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
        
        
zone_data['Diaper Ach%'] = zone_data.apply(lambda row: safe_divide(row['Diaper_ach'], row['FA_diaper_target']), axis=1)
zone_data['Napkin Ach%'] = zone_data.apply(lambda row: safe_divide(row['Napkin_ach'], row['FA_napkin_target']), axis=1)
zone_data['UTC%'] = zone_data.apply(lambda row: safe_divide(row['UTC'], row['Count of Outlets']), axis=1)
zone_data['UPC%'] = zone_data.apply(lambda row: safe_divide(row['UPC'], row['Count of Outlets']), axis=1)
zone_data['Average Order value'] = zone_data.apply(lambda row: safe_divide(row['Total Achivement'], row['PC']), axis=1)
zone_data['TotalAch%'] = zone_data.apply(lambda row: safe_divide(row['Total Achivement'], row['FA_total_target']), axis=1)
zone_data['DRR'] = zone_data.apply(lambda row: safe_divide(row['Total Achivement'], number_of_working_days), axis=1)
zone_data['napkin_eco%'] = zone_data.apply(lambda row: safe_divide(row['Napkin_eco'], row['Count of Outlets']), axis=1)
zone_data['diaper_eco%'] = zone_data.apply(lambda row: safe_divide(row['Diaper_eco'], row['Count of Outlets']), axis=1)                                                        
zone_data['Average Order value'] *= 100000
zone_data['Diaper Ach'] = zone_data['Diaper_ach'].round(2)
zone_data['Napkin Ach'] = zone_data['Napkin_ach'].round(2)
zone_data['Napkin Ach%'] = zone_data['Napkin Ach%'].round(2)
zone_data['Diaper Ach%'] = zone_data['Diaper Ach%'].round(2)
zone_data['UTC%'] = zone_data['UTC%'].round(2)
zone_data['UPC%'] = zone_data['UPC%'].round(2)
zone_data['TotalAch%'] = zone_data['TotalAch%'].round(2)
zone_data['DRR'] = zone_data['DRR'].round(2)
zone_data['RRR'] = zone_data['RRR'].round(2)
zone_data['napkin_eco%'] = zone_data['napkin_eco%'].round(2)
zone_data['diaper_eco%'] = zone_data['diaper_eco%'].round(2)
zone_data['Average Order value'] = zone_data['Average Order value'].round(2)
weekdays = count_working_days_until_today()-1



Regions_data = sr_summary.groupby(['Regions']).agg({
    'Erp Code': 'nunique',  # Count unique salesmen
    'Order Date_x': 'sum',
    'Count of Outlets': 'sum' ,
    'TC': 'sum' ,
    'UTC': 'sum' ,
    'PC': 'sum' ,
    'UPC': 'sum' ,
    'LPC_value': 'mean' ,
    'Total Achivement': 'sum',
    'Diaper_ach': 'sum',
    'Napkin_ach': 'sum',
    'Napkin_eco': 'sum',
    'Diaper_eco': 'sum',
    'dc_eco' : 'sum',
    'ns_eco': 'sum',
    'ultra_eco': 'sum',
    'ultrapremium_eco': 'sum',
    'DRR': 'sum' ,
    'RRR': 'sum' 
}).reset_index()


target_region = target_region.rename(columns={'Target': 'FA_napkin_target'})
target_region = target_region.rename(columns={'Napkin Target': 'Primary_napkin_target'})
target_region = target_region.rename(columns={'Diaper Target': 'Primary_diaper_target'})
target_region = target_region.rename(columns={'Total Target': 'Primary_total_target'})
target_region = target_region.rename(columns={'Napkin Target.1': 'FA_napkin_target'})
target_region = target_region.rename(columns={'Diaper Target.1': 'FA_diaper_target'})
target_region = target_region.rename(columns={'Total Target .1': 'FA_total_target'})
target_region = target_region.rename(columns={'Napkin Target.2': 'SNS_napkin_target'})
target_region = target_region.rename(columns={'Diaper Target.2': 'SNS_diaper_target'})
target_region = target_region.rename(columns={'Total Target .2': 'SNS_total_target'})

target_region= target_region[['Region','FA_napkin_target', 'FA_diaper_target',

       'FA_total_target','SNS_napkin_target', 'SNS_diaper_target',

       'SNS_total_target']]
target_region = target_region.rename(columns={'Region': 'Regions'})
target_region['Regions'] = target_region['Regions'].str.replace(' Total', '', regex=False)
 
Regions_data = pd.merge(Regions_data, target_region, on='Regions', how='left')

def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
        
        
Regions_data['Diaper Ach%'] = Regions_data.apply(lambda row: safe_divide(row['Diaper_ach'], row['FA_diaper_target']), axis=1)
Regions_data['Napkin Ach%'] = Regions_data.apply(lambda row: safe_divide(row['Napkin_ach'], row['FA_napkin_target']), axis=1)
Regions_data['UTC%'] = Regions_data.apply(lambda row: safe_divide(row['UTC'], row['Count of Outlets']), axis=1)
Regions_data['UPC%'] = Regions_data.apply(lambda row: safe_divide(row['UPC'], row['Count of Outlets']), axis=1)
Regions_data['Average Order value'] = Regions_data.apply(lambda row: safe_divide(row['Total Achivement'], row['PC']), axis=1)
Regions_data['TotalAch%'] = Regions_data.apply(lambda row: safe_divide(row['Total Achivement'], row['FA_total_target']), axis=1)
Regions_data['DRR'] = Regions_data.apply(lambda row: safe_divide(row['Total Achivement'], number_of_working_days), axis=1)
Regions_data['napkin_eco%'] = Regions_data.apply(lambda row: safe_divide(row['Napkin_eco'], row['Count of Outlets']), axis=1)
Regions_data['diaper_eco%'] = Regions_data.apply(lambda row: safe_divide(row['Diaper_eco'], row['Count of Outlets']), axis=1)                                                        
Regions_data['Average Order value'] *= 100000
Regions_data['Diaper Ach'] = Regions_data['Diaper_ach'].round(2)
Regions_data['Napkin Ach'] = Regions_data['Napkin_ach'].round(2)
Regions_data['Napkin Ach%'] = Regions_data['Napkin Ach%'].round(2)
Regions_data['Diaper Ach%'] = Regions_data['Diaper Ach%'].round(2)
Regions_data['UTC%'] = Regions_data['UTC%'].round(2)
Regions_data['UPC%'] = Regions_data['UPC%'].round(2)
Regions_data['TotalAch%'] = Regions_data['TotalAch%'].round(2)
Regions_data['DRR'] = Regions_data['DRR'].round(2)
Regions_data['RRR'] = Regions_data['RRR'].round(2)
Regions_data['napkin_eco%'] = Regions_data['napkin_eco%'].round(2)
Regions_data['diaper_eco%'] = Regions_data['diaper_eco%'].round(2)
Regions_data['Average Order value'] = zone_data['Average Order value'].round(2)
weekdays = count_working_days_until_today()-1

sr_summary=sr_summary[['Regions',	'Zone',	'Territory',	'User',	'Erp Code',		'User Status',	'Date of Joining','Order Date_x',	'Count of Outlets',	'TC',	'UTC',	'UTC%',	'PC',	'UPC',	'UPC%',	'LPC_value','Average Order value',	'FA_napkin_target',	'Napkin_ach',	'Napkin Ach%',	'FA_diaper_target',	'Diaper_ach',	'Diaper Ach%',	'FA_total_target',	'Total Achivement',	'TotalAch%',	'DRR',	'RRR',	'Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
territory_data=territory_data[['Regions',	'Zone',	'Territory',	'Erp Code',	'Order Date_x',	'Count of Outlets',	'TC',	'UTC',	'UTC%',	'PC',	'UPC',	'UPC%',	'LPC_value','Average Order value',	'FA_napkin_target','SNS_napkin_target',	'Napkin Ach',	'Napkin Ach%',	'FA_diaper_target','SNS_diaper_target',	'Diaper_ach',	'Diaper Ach%',	'FA_total_target','SNS_total_target','Total Achivement',	'TotalAch%',	'DRR',	'RRR',	'Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
zone_data=zone_data[['Regions',	'Zone',	'Erp Code',	'Order Date_x',	'Count of Outlets',	'TC',	'UTC',	'UTC%',	'PC',	'UPC',	'UPC%','LPC_value',	'Average Order value',	'FA_napkin_target',	'SNS_napkin_target','Napkin Ach',	'Napkin Ach%',	'FA_diaper_target','SNS_diaper_target',	'Diaper_ach',	'Diaper Ach%',	'FA_total_target','SNS_total_target',	'Total Achivement',	'TotalAch%',	'DRR',	'RRR','Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
Regions_data=Regions_data[['Regions',	'Erp Code',	'Order Date_x',	'Count of Outlets',	'TC',	'UTC',	'UTC%',	'PC',	'UPC',	'UPC%','LPC_value',	'Average Order value',	'FA_napkin_target'	,'SNS_napkin_target',	'Napkin Ach',	'Napkin Ach%',	'FA_diaper_target','SNS_diaper_target',	'Diaper_ach',	'Diaper Ach%',	'FA_total_target','SNS_total_target',	'Total Achivement',	'TotalAch%',	'DRR',	'RRR','Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
region_eco=Regions_data[['Regions',	'Erp Code',	'Count of Outlets','Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
zone_eco=zone_data[['Regions',	'Zone',	'Erp Code',	'Count of Outlets',	'Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]
beat_summary=beat_summary[['Regions',	'Zone',	'Territory',	'User',	'Erp Code',		'Beat Erp Id',	'Beats','Order Date',	'Count of Outlets',	'TC',	'UTC',	'UTC%',	'PC',	'UPC',	'UPC%',	'Average Order value',	'Napkin_ach',	'Diaper_ach',	'Net Value',	'DRR',		'Napkin_eco',	'napkin_eco%',	'Diaper_eco',	'diaper_eco%',	'dc_eco',	'ns_eco',	'ultra_eco',	'ultrapremium_eco']]


In [11]:
import xlwings as xw
workbook=xw.Book(r'C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\SR Daily Performance Tracker Automated.xlsx') 
region = xw.sheets('Region summary') 
zone = xw.sheets('Zone summary')
terrirtory = xw.sheets('Territory Summary')
sr_summary1 = xw.sheets('SR Summary')
beat_summary1= xw.sheets('Beat Summary')

sr_summary=sr_summary.reset_index() 
territory_data= territory_data.reset_index() 
zone_data= zone_data.reset_index() 
Regions_data=Regions_data.reset_index()
beat_summary2=beat_summary.reset_index()
zone_eco= zone_eco.reset_index() 
region_eco=region_eco.reset_index()

sr_summary1.range('A6:BP500000').value = ""
terrirtory.range('A6:BP500000').value = ""
zone.range('A6:BP500000').value = ""
region.range('A6:BP500000').value = ""
beat_summary1.range('A6:BP500000').value = ""

sr_summary1.range('A6').value = [sr_summary.columns.tolist()]
sr_summary1.range('A6').value = sr_summary.values

sr_summary1.range('A6').value = sr_summary.values
terrirtory.range('A6').value = [territory_data.columns.tolist()]
terrirtory.range('A6').value = territory_data.values
zone.range('A6').value = [zone_data.columns.tolist()]
zone.range('A6').value = zone_data.values
region.range('A6').value = [Regions_data.columns.tolist()]
region.range('A6').value = Regions_data.values

zone.range('AQ6').value = [zone_eco.columns.tolist()]
zone.range('AQ6').value = zone_eco.values
region.range('AQ6').value = [region_eco.columns.tolist()]
region.range('AQ6').value = region_eco.values

beat_summary1.range('A6').value = [beat_summary2.columns.tolist()]
beat_summary1.range('A6').value = beat_summary2.values
workbook.api.RefreshAll()


workbook.save()

import xlwings as xw
from PIL import ImageGrab
import time
import os

# Define the path where the image will be saved
path = r'C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images'

# Open the workbook and select the sheet
workbook_path = r'C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\SR Daily Performance Tracker Automated.xlsx'
workbook = xw.Book(workbook_path)
sheet = workbook.sheets['Region Summary']

# Define the range to copy
copy_range = "A1:AA13"

def export_range_as_image(sheet, copy_range, image_name, save_path):
    # Select the range
    rng = sheet.range(copy_range)
    
    # Copy the range as a picture
    rng.api.CopyPicture(Appearance=1, Format=2)
    
    # Grab the image from the clipboard
    img = ImageGrab.grabclipboard()
    
    # Check if the image was successfully grabbed
    if img:
        # Define the full path for the image
        full_image_path = os.path.join(save_path, f"{image_name}.png")
        
        # Save the image
        img.save(full_image_path)
        print(f"Image saved successfully at {full_image_path}")
    else:
        print("No image found in clipboard")

# Call the function to export the image
export_range_as_image(sheet, copy_range, "Region", path)


copy_range = "AQ1:BI13"

def export_range_as_image(sheet, copy_range, image_name, save_path):
    # Select the range
    rng = sheet.range(copy_range)
    
    # Copy the range as a picture
    rng.api.CopyPicture(Appearance=1, Format=2)
    
    # Grab the image from the clipboard
    img = ImageGrab.grabclipboard()
    
    # Check if the image was successfully grabbed
    if img:
        # Define the full path for the image
        full_image_path = os.path.join(save_path, f"{image_name}.png")
        
        # Save the image
        img.save(full_image_path)
        print(f"Image saved successfully at {full_image_path}")
    else:
        print("No image found in clipboard")

# Call the function to export the image
export_range_as_image(sheet, copy_range, "Region_eco", path)


sheet = workbook.sheets['Zone Summary']

# Define the range to copy
copy_range = "A1:AB32"

def export_range_as_image(sheet, copy_range, image_name, save_path):
    # Select the range
    rng = sheet.range(copy_range)
    
    # Copy the range as a picture
    rng.api.CopyPicture(Appearance=1, Format=2)
    
    # Grab the image from the clipboard
    img = ImageGrab.grabclipboard()
    
    # Check if the image was successfully grabbed
    if img:
        # Define the full path for the image
        full_image_path = os.path.join(save_path, f"{image_name}.png")
        
        # Save the image
        img.save(full_image_path)
        print(f"Image saved successfully at {full_image_path}")
    else:
        print("No image found in clipboard")

# Call the function to export the image
export_range_as_image(sheet, copy_range, "Zone", path)

copy_range = "AQ1:BI32"

def export_range_as_image(sheet, copy_range, image_name, save_path):
    # Select the range
    rng = sheet.range(copy_range)
    
    # Copy the range as a picture
    rng.api.CopyPicture(Appearance=1, Format=2)
    
    # Grab the image from the clipboard
    img = ImageGrab.grabclipboard()
    
    # Check if the image was successfully grabbed
    if img:
        # Define the full path for the image
        full_image_path = os.path.join(save_path, f"{image_name}.png")
        
        # Save the image
        img.save(full_image_path)
        print(f"Image saved successfully at {full_image_path}")
    else:
        print("No image found in clipboard")

# Call the function to export the image
export_range_as_image(sheet, copy_range, "zone_eco", path)


# Introduce a delay to ensure the image is saved
time.sleep(3)

# Close the workbook without saving changes
workbook.close()

Image saved successfully at C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images\Region.png
Image saved successfully at C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images\Region_eco.png
Image saved successfully at C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images\Zone.png
Image saved successfully at C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images\zone_eco.png


In [12]:
import smtplib
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import mimetypes
import os
from datetime import datetime
 
# Get today's date
today_date = datetime.now().strftime('%Y-%m-%d')
 
# Email components
region = '<img src="cid:Region"></img><br/><br/>'
zone = '<img src="cid:Zone"></img><br/><br/>'
region_eco = '<img src="cid:Region_eco"></img><br/><br/>'
zone_eco = '<img src="cid:zone_eco"></img><br/><br/>'

Image1 = '<img src="cid:region"></img><br/><br/>'
Image2 = '<img src="cid:region_eco"></img><br/><br/>'
Image3 = '<img src="cid:zone"></img><br/><br/>'
Image4 = '<img src="cid:zone_eco"></img><br/><br/>'
 
text0 = "<div style='color:red'>  </div> <br/>"
text1 = "Hi All,<br/> <br/> PFA SR Daily Performance Tracker.<br/><br/>"
text2 = "Note:<br/> <br/> 1. This tracker is for current active SRs.<br/> <br/> 2. For clarity, targets at Region, Zone and Territory level are same as shared in final target file, irrespective of SRs.<br/><br/>"
regards = '<br/> <br/> <br/> <br/> Regards <br/> Niine Analytics Team'
body = text0 + text1 + text2 + region + region_eco + zone + zone_eco + regards
 
subject = f'SR Daily Performance Tracker|| Auto || {today_date}'
 
rec_To_list = [
'rohit.shahi@niine.com',
'manoj.mishra@niine.com',
'kuldeep.singh@niine.com',
'sanjay.kumar@niine.com',
'rupal.agarwal@niine.com',
'gopal.prasad1@niine.com',
'gaurav.seth@niine.com',
'sachin.bhalme@niine.com',
'ajay.maan@niine.com',
'vikas.singh@niine.com',
'ravi.nigam@niine.com',
'prakhar.shukla@niine.com',
'kapil.tyagi@niine.com',
'mohamed.noor@niine.com',
'pankaj.bhagat@niine.com',
'ram.sanehi@niine.com',
'sarabjeet.singh@niine.com',
'g.srikanth@niine.com',
'amit.das@niine.com',
'amit.kumar@niine.com',
'anutosh.ranjan@niine.com',
'dileep.dubey@niine.com',
'vinay.thacker@niine.com',
'gedela.srinivas@niine.com',
'avinshtiwari@niine.com',
'yugank.mishra@niine.com',
'abhishek.das03@niine.com',
'udeshya.gupta@niine.com',
'surya.prakash@niine.com',
'kalu@niine.com',
'deepak.patil@niine.com',
'kaushal.kumar1@niine.com',
'santosh.dhal@niine.com',
'rajkumar.halder@niine.com',
'praloy.sen@niine.com',
'pankaj.jha@niine.com',
'pawan.sharma@niine.com',
'ajay.jha@niine.com',
'amit.k@niine.com',
'santanu.sen@niine.com',
'satyapal.sharma@niine.com',
'ketan.vaidya@niine.com',
'amitava.ghosh@niine.com',
'saikat.chakraborty@niine.com',
'purushottam.mishra@niine.com',
'motiraj.tamang@niine.com',
'pankaj.thakur@niine.com',
'sandeep.singh@niine.com',
'umesh.upmanyu@niine.com',
'priyank.roy@niine.com',
'bhupesh.mishra@niine.com',
'arpan.mondal@niine.com',
'op.pandey@niine.com ',
'shankar.gurnani@niine.com  ',
'shashikant.kori@niine.com',
'saurabh.kumar@niine.com',
'anurag.singh@niine.com',
'bhikam.singh@niine.com ',
'tripurari.tripathi@niine.com',
'koushik.roy@niine.com',
'ashish.kumar@niine.com',
'dhananjay.joshi@niine.com',
'ashim.saha@niine.com',
'naseem.jawaid@niine.com',
'sujith.sripadha@niine.com',
'hitesh.sahu@niine.com',
'joydeep.biswas@niine.com',
'kulveer.rastogi@niine.com',
'sudip.banerjee@niine.com',
'ganpat.rathore@niine.com',
'vikas.jayaswal@niine.com',
'ratnesh.dwivedi@niine.com',
'anil.mishra@niine.com',
'basant.singh@niine.com',
'jitendra.yadav@niine.com',
'bhupesh@niine.com ',
'dhyanpal.singh@niine.com',
'vishal.bhan@niine.com',
'ravi.ranjan@niine.com',
'niraj.upadhyay@niine.com',
'prasanta.mitra@niine.com',
'suresh.kumar@niine.com',
'tanvir.pathan@niine.com',
'prasun.bhowmik@niine.com',
'hamid.ansari@niine.com',
'lokendra.singh@niine.com',
'shailendra.kumar@niine.com',
'Agastya.singh@niine.com',
'Shreemala.Dhayal@niine.com'
]
to_email_list = ', '.join(rec_To_list)
 
rec_CC_list = [ 
   'ameya.dangi@niine.com','Shreemala.Dhayal@niine.com','gaurav@niine.com','krishan.malav@niine.com','Akshay.Sundareshan@niine.com','Agastya.singh@niine.com'
]
to_CC_list = ', '.join(rec_CC_list)
 
# File and image paths
filename = r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\SR Daily Performance Tracker Automated.xlsx"
image_path = r"C:\Users\Agastya Singh\OneDrive - Niine Pvt. Ltd\Desktop\SR Daily tracker\images" # Update with the actual path to the images
 
def send_email(subject, body, to_email_list, filename):
    to_emails = to_email_list
    to_CC = to_CC_list
    message = body
 
    # Set up the SMTP server
    s = smtplib.SMTP(host='smtp.outlook.office365.com', port=587) 
    s.starttls()
    s.login('ram.kishor@niine.com', 'Niine@2024')  # Use environment variable or secure method
 
    # Create the email
    msg = MIMEMultipart()
    msg['From'] = 'ram.kishor@niine.com'
    msg['To'] = to_emails
    msg['CC'] = to_CC
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))
 
    def attach_image(image_name):
        with open(os.path.join(image_path, image_name + '.png'), 'rb') as fp:
            msg_image = MIMEImage(fp.read())
            msg_image.add_header('Content-ID', f'<{image_name}>')
            msg.attach(msg_image)
 
    # Attach images
    attach_image('region')
    attach_image('region_eco')
    attach_image('zone')
    attach_image('zone_eco')
 
    # Attach file
    ctype, encoding = mimetypes.guess_type(filename)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"
    maintype, subtype = ctype.split("/", 1)
    with open(filename, "rb") as fp:
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        encoders.encode_base64(attachment)
        attachment.add_header("Content-Disposition", "attachment", filename=os.path.basename(filename))
        msg.attach(attachment)
 
    # Send the email
    s.send_message(msg)
    s.quit()
 
print('Sending Mail')
send_email(subject, body, to_email_list, filename)
print('SR daily tracker Sent')
 
# Optional: Close any open Excel processes
os.system("TASKKILL /F /IM excel.exe")

Sending Mail
SR daily tracker Sent


0